## 1. Построение мета-игры 

Добавим параметры в наши исходные игры. Начнем с игры **XA** (немного округлим значения, чтобы матрица игры не была слищком сложной)

$$
G_{XA} = 
\left(
\begin{matrix}
   3x & 6x & x \\
   5x & 5x & 5x  \\
   6x & 6x & 4x  
\end{matrix}
\right)
$$

Ограничим прибыль за месяц для обеих кофейнь: $x \leq 36000$ 

Тогда прибыль за весь период продаж обеих кофейнь будет варьироваться от нуля до максимальной прибыли, которая возможна, т.е $$0 \leq \sum x\leq 108 000$$

В последний месяц продажи кофе падают, так как наступае зима и "кофе с собой" становится менее привлекательным для потребятиля.
Чтобы како-то исправить положение, наши предприятия решают выпускать кофе с корицей (в надежде на то, что он будет более привлекательным в декабре). Но количество таких чашек ограничено, т.к. достать все составляющие зимой достаточно сложно.


Максимальное количество новых чашек в месяц: $T = 500$ штук. Цена одной такой чашки - 250р (при себестоимости в 150р -> доход $p_x = 100$р).

Сколько же таких чашек сможет продать наша кофейня с учетом ограничения на прибыль?
$$T = 3600$$ чашек.

Тогда прибыль нашей кофейни в последний месяц будем искать по закону:
$$
P = (3x - \sum x) + cTp_x, \ где
$$
* c - значение от 0 до 1, в которое мы должны увеличить варку нового кофе
* M - максимальное количество новых чашек кофе
* $p_x$ - прибыль от продажи новой чашки кофе

Дополнительно введем льготы малому бизнесу, добавляя к прибыли выплаты: $r$ не более 3% от прибилы.

Учитывая все ограничения, получим матрицу игры
$$
G_{XA} = 
\left(
\begin{matrix}
   3x(1 + r) & 6x(1 + r) & x(1 + r) \\
   5x(1 + r) & 5x(1 + r) & 5x(1 + r)  \\
   6x(1 + r) + 5000c(1 + r) & 6x(1 + r) + 5000c(1 + r) & 4x(1 + r) + 5000c(1 + r)
\end{matrix}
\right)
$$

Аналогично составим матрицы оставшихся двух игр:
$$
G_{XB} = 
\left(
\begin{matrix}
   x(1 + r) & 2x(1 + r) \\
   x(1 + r) & x(1 + r)  \\
   1.25x(1 + r) + 5000c(1 + r) & 1.25x(1 + r) + 5000c(1 + r)  \\
\end{matrix}
\right)
$$

$$
G_{XC} = 
\left(
\begin{matrix}
   1.3x(r + 1) & 2x(r + 1) & x(r + 1) \\
   1.5x(r + 1) + 5000c(r + 1)& 1.5x(r + 1) + 5000c(1 + r) & 1.5x(r + 1) + 5000c(1 + r)  \\
\end{matrix}
\right)
$$

## 2. Вектор параметров

Векто параметров в нашем случае имеет вид:
$$
M = [x, c, r]
$$

## 3. Невязка

Допустим с текущим конкретным вектором $M_0$ у вас получается игра $G_0$ и стратегии $A,B$
$$
F(M_0) = \sum a_i b_j G[i,j](M_0)
$$

$$
\sigma_1 = [ \sigma, 0,0]\\
\sigma_2 = [0, \sigma,0 ]\\
\sigma_3 = [0,0,\sigma ]
\\
\sigma = 0.02
$$

Далее высчитываем невязку: 
$$
\Delta_1 = F(M_0 + \sigma_1) - F(M_0)
$$
И вектор невязки:
$$
\Delta = [\Delta_1,\Delta_2,\Delta_3]
$$

## 4. Поиск невязки на python

Возьмем в качестве начального вектора
$$
M = [30000, 0.5, 0.01]
$$

### XA

In [102]:
from scipy.optimize import linprog
import numpy as np
import math

def get_game(x_coefs: np.ndarray, M: list) -> list:
    res = np.zeros(x_coefs.shape)
    for i in range(x_coefs.shape[0]):
        for j in range(x_coefs.shape[1]):
            res[i][j] = M[0] * x_coefs[i][j] * (1 + M[2])
    for j in range(res.shape[1]):
        res[res.shape[0] - 1][j] += 5000 * M[1]*(1 + M[2])
    return res

def solve_game(x_coefs: np.ndarray, M: list, sigma: float):
    game_m = get_game(x_coefs, M)
    print("Платежная матрица:\n{0}".format(game_m))

    #Теперь посмотрим, как влияет сигма на нашу матрицу.
    # Для этого будем добавлять сигму в входным параметрам и смотреть, как отклоняется решение игры
    # Когда мы прибавляем сигму к x, то умножаем ее на 100 из-за большой разницы в порядках чисел
    simed_games = []
    simed_games.append(-get_game(x_coefs, [x + sigma * 100, c, r]))
    simed_games.append(-get_game(x_coefs, [x, c + sigma, r]))
    simed_games.append(-get_game(x_coefs, [x, c, r + sigma]))

    simga_impacts = []

    f = np.ones(x_coefs.shape[1])
    b = (-1) * np.ones(x_coefs.shape[0])
    for simed_game in simed_games:
        simga_impacts.append(linprog(f, simed_game, b))

    F_M0 = 1 / linprog(f, game_m, b).fun

    deltas = []
    for simga_impact in simga_impacts:
        deltas.append(round(1 / simga_impact.fun - F_M0))

    print("Невязка: {0}".format(deltas))

    normalized_deltas = 0
    for delta in deltas:
        normalized_deltas += delta ** 2
    normalized_deltas = math.sqrt(normalized_deltas)

    print("Нормализованный вектор невязки: {0}".format(np.array(deltas) / normalized_deltas))

In [103]:
T = 500
x = 30000
c = 0.5
r = 0.01
sigma = 0.2

M = [x, c, r]
x_coefs = np.array([[3, 6, 1],
                  [5, 5, 5],
                  [6, 6, 4]])

solve_game(x_coefs, M, sigma)

Платежная матрица:
[[ 90900. 181800.  30300.]
 [151500. 151500. 151500.]
 [184325. 184325. 123725.]]
[[ -90960.6 -181921.2  -30320.2]
 [-151601.  -151601.  -151601. ]
 [-184446.2 -184446.2 -123805.8]]
[[ -90900. -181800.  -30300.]
 [-151500. -151500. -151500.]
 [-185335. -185335. -124735.]]
[[-108900. -217800.  -36300.]
 [-181500. -181500. -181500.]
 [-220825. -220825. -148225.]]
Невязка: [101354, 101253, 131253]
Нормализованный вектор невязки: [0.52163897 0.52111915 0.67552025]


Сформируем профиль решения мета-игры (вектор-градиент спуска функции стоимости игры по ее профилю), описывав его словестно.
Мы видим, что по всем трем параметрам наш вектор положителен.
 * Первая координата положительна, т.к. увеличение прибыли для нас явно будет полезно
 * Вторая координата положительна, т.к. увеличение объемов производства влечет за собой увеличение прибыли
 * Третья координата положительна, т.к. с увеличением льгот (надбавки на прибыль), наша выгода растет
 

### XB

In [106]:
T = 500
x = 30000
c = 0.5
r = 0.01
sigma = 0.2

M = [x, c, r]
x_coefs = np.array([[1.3, 1, 2],
                  [1.5, 1.5, 1.5]])
solve_game(x_coefs, M, sigma)

Платежная матрица:
[[39390. 30300. 60600.]
 [47975. 47975. 47975.]]
[[-39416.26 -30320.2  -60640.4 ]
 [-48005.3  -48005.3  -48005.3 ]]
[[-39390. -30300. -60600.]
 [-48985. -48985. -48985.]]
[[-47190. -36300. -72600.]
 [-57475. -57475. -57475.]]
Невязка: [14509, 15489, 23979]
Нормализованный вектор невязки: [0.45309383 0.48369773 0.74882741]


В этой игре имеют место те же выводы, что мы сделали для матрицы **XA**

### XC

In [ ]:
T = 500
x = 30000
c = 0.5
r = 0.01
sigma = 0.2

M = [x, c, r]
x_coefs = np.array([[1.3, 2, 1],
                  [1, 1],
                  [1.25, 1.25]])
solve_game(x_coefs, M, sigma)

В этой игре имеют место те же выводы, что мы сделали для матрицы **XA**